<h3>Simulating 100k+ battle openings for the Pokemon Trading Card Game Pocket game, using the current most popular meta deck: <strong>Pikachu EX.</strong></h3> 
The goal of this simulation is to answer the following questions:
<ul>
    <li>When is the deck able to pull off its primary game plan?</li>
    <li>What is the most consistent build?</li>
    <li>How much of a difference does going first and going second make?</li>
    <li>How many basic cards is the ideal number of basic cards?</li>
</ul>

<h3>The game board is as follows:</h3>

       []       One active slot   
    [] [] []    Three bench slots 



<h4>What are the rules in play?</h4>
<ul>
    <li>You draw 5 cards at the start; at least one basic/Pikachu EX will be drawn on your first turn.</li>
    <li>You can place as many basic/Pikachu EX cards during a turn as you want.</li>
    <li>You start drawing 1 energy per turn after your first turn; you can place one energy on a basic/Pikachu EX per turn.</li>
    <li>You can attack with Pikachu EX once it gets 2 energy; Pikachu EX does 30 damage for every filled bench slot.</li>
    <li>For the sake of simplifying the simulation, 1 energy is required to swap from the bench slot to the active slot</li>
</ul>

<h4>What is the primary game plan of the Pikachu EX deck?</h4>
<ol>
    <li>Have Pikachu EX in the active slot.</li>
    <li>Fill your bench with as many basic/Pikachu EX cards as possible.</li>
    <li>Attach 2 energy to Pikachu EX and attack the opponent.</li>
</ol>

<h4>Caveats?</h4>
If Pikachu EX is not drawn on turn one, your primary game plan will be delayed. Extra turns may be needed to swap a buffer basic from the active slot to a drawn Pikachu EX on one of the three bench slots.

<h4>What does each card mean?</h4>

>PIKA_EX - your primary game plan, Pikachu EX<br>
>BASIC - cards that you need to fill your bench with<br>
>P_BALL - use to draw one random BASIC/PIKA_EX card to hand<br>
>P_RE - use to draw two cards at random; only one P_RE can be used per turn<br>
>X_SP - use to make swapping a card from your bench to active slot not require energy use<br>
>OTHER - cards which will not affect your primary game plan

In [2]:
from enum import Enum
from random import shuffle, randint
import pandas as pd
import random
import sys

class Card(Enum):
    PIKA_EX = 1
    BASIC = 2
    P_BALL = 3
    P_RE = 4
    X_SP = 5
    OTHER = 6

class BoardState:
    active_slot = []
    bench = []
    energy = False
    basic_energy_count = 0
    pika_energy_count = 0
    damage_counter = 0

    def reset():
        BoardState.active_slot = []
        BoardState.bench = []
        BoardState.energy = False
        BoardState.basic_energy_count = 0
        BoardState.pika_energy_count = 0
        BoardState.damage_counter = 0

class Simulation: 
    
    #Game start: initiate hand and deck shuffled
    def __init__(self, turn, basic_count):
        self.basic_count = basic_count
        self.turn = turn
        self.hand = []
        self.deck = [Card.PIKA_EX] * 2 + \
                    [Card.P_BALL] * 2 + \
                    [Card.P_RE] * 2 + \
                    [Card.X_SP] * 2 + \
                    [Card.BASIC] * basic_count + \
                    [Card.OTHER] * (12 - basic_count)
        shuffle(self.deck)

    def display_bs(self):
        print(BoardState.active_slot)
        print(BoardState.bench)

    #Remove a card from deck and place it in your hand
    def draw_card(self, times=1):
        for card in range(times):
            try:
                self.hand.append(self.deck.pop())
            except:
                #print('Nothing in deck to draw!')
                pass

    def turn_0(self):

        #Put PIKA_EX or BASIC into the active slot if empty
        if not BoardState.active_slot:
            if Card.PIKA_EX in self.hand:
                BoardState.active_slot = Card.PIKA_EX
                self.hand.remove(Card.PIKA_EX)
            else:
                BoardState.active_slot = Card.BASIC
                self.hand.remove(Card.BASIC)
 
        #Fill the bench with PIKA_EX/BASIC up to 3
        while len(BoardState.bench) < 3:
            if Card.PIKA_EX in self.hand:
                BoardState.bench.append(Card.PIKA_EX)
                self.hand.remove(Card.PIKA_EX)
            elif Card.BASIC in self.hand and (BoardState.bench + [BoardState.active_slot]).count(Card.BASIC) < 3:   #Won't fill the board with all basics (leaves room for PIKA_EX)
                BoardState.bench.append(Card.BASIC)
                self.hand.remove(Card.BASIC)
            else:
                break

        return len(BoardState.bench + [BoardState.active_slot])

        #self.display_bs()

    def player_turn(self):
        
        self.draw_card(1)

        if ((self.deck + BoardState.bench + [BoardState.active_slot]).count(Card.PIKA_EX) >= 3):
            print('eek2')
            sys.exit()

        #Energy generation; 1 energy available per turn, with the exception of player turn 1
        if self.turn != 0:
            BoardState.energy = True

        #Play available card draw: first P_RE then P_BALL to maximize chances of pulling PIKA_EX/filling the board
        try:
            if Card.P_RE in self.hand:
                #print('Playing Professors Research')
                self.hand.remove(Card.P_RE)
                self.draw_card(2)
            while Card.P_BALL in self.hand:
                #print('Playing Pokeball')
                self.hand.remove(Card.P_BALL)
                index_choice = random.choice([i for i, x in enumerate(self.deck) if x == Card.BASIC or x == Card.PIKA_EX])
                self.hand.append(self.deck[index_choice])
                self.deck.remove(self.deck[index_choice])
        except:
            #print('Nothing in deck to draw!')
            pass

        #Play newly drawn PIKA_EX or BASIC cards
        while len(BoardState.bench) < 3:
            if Card.PIKA_EX in self.hand:
                BoardState.bench.append(Card.PIKA_EX)
                self.hand.remove(Card.PIKA_EX)
            elif Card.BASIC in self.hand and (BoardState.bench + [BoardState.active_slot]).count(Card.BASIC) < 3:   #Won't fill the board with all basics (leaves room for PIKA_EX)
                BoardState.bench.append(Card.BASIC)
                self.hand.remove(Card.BASIC)
            else:
                break

        #Engage PIKA_EX into active slot if not already, by means of X_SP or existing energy count on a BASIC
        if BoardState.active_slot != Card.PIKA_EX and Card.PIKA_EX in BoardState.bench:
            if Card.X_SP in self.hand:
                #print('Playing X-Speed')
                self.hand.remove(Card.X_SP)
                BoardState.active_slot = Card.PIKA_EX
                BoardState.bench.remove(Card.PIKA_EX)
                BoardState.bench.append(Card.BASIC)
            elif BoardState.basic_energy_count > 0:
                BoardState.basic_energy_count = 0
                BoardState.active_slot = Card.PIKA_EX
                BoardState.bench.remove(Card.PIKA_EX)
                BoardState.bench.append(Card.BASIC)
        
        #Place energy on whatever is in the active slot, UNLESS it is a BASIC and already has one
        while BoardState.energy == True:
            if Card.PIKA_EX == BoardState.active_slot:
                BoardState.pika_energy_count += 1
            elif Card.BASIC == BoardState.active_slot:
                BoardState.basic_energy_count += 1
            BoardState.energy = False

        #Attack when PIKA_EX reaches 2 energy
        if BoardState.pika_energy_count >= 2:
            BoardState.damage_counter += len(BoardState.bench) * 30

        #self.display_bs()
        #print('Pika energy count: ', BoardState.pika_energy_count)
        #print('Basic energy count ', BoardState.basic_energy_count)
        #print('Damage: ', BoardState.damage_counter)

        #End turn
        if self.turn == 0:
            self.turn += 2
            return 1, len(BoardState.bench + [BoardState.active_slot]), BoardState.pika_energy_count, BoardState.damage_counter
        else:
            self.turn += 1
            return self.turn-1, len(BoardState.bench + [BoardState.active_slot]), BoardState.pika_energy_count, BoardState.damage_counter

    

    def runSimulation(self):

        #Create dataframe
        df = pd.DataFrame(columns=['turn','board_mons','pika_energy','damage_counter'])

        #Draw your starting hand, if PIKA_EX or BASIC isn't drawn, redraw your hand
        while not (Card.PIKA_EX in self.hand or Card.BASIC in self.hand):
            self.__init__(self.turn, self.basic_count)
            self.draw_card(5)

        #Conduct first player turn, fill dataframe with turn 0 info
        df.loc[0] =[0, self.turn_0(), 0, 0]
        self.turn_0()

        #Play through 12 turns
        for turn in range(1,12):
            df.loc[turn] = self.player_turn()
        
        return df

In [293]:
#Starting tails (1), basic count of 4 (4)
BoardState.reset()
print(Simulation(1, 4).runSimulation())

   turn  board_mons  pika_energy  damage_counter
0     0           1            0               0
1     1           3            0               0
2     2           4            1               0
3     3           4            2              90
4     4           4            3             180
5     5           4            4             270
6     6           4            5             360
7     7           4            6             450


So what are we looking for here?<br>
We'll be looking for the moment in the simulation that the primary game plan is achieved. In this case above, this would be on turn 3. A full board and 2 energy on Pikachu_EX results in 90 full damage. For future cases, since the damage counter is cumulative damage, we'll be measuring the first turn that 90 damage *or above* is reached as succeeding in accomplishing the primary game plan.<br><br>

Now, we'll run the simulation many times, with several different variables, only gathering the turn when the primary game plan is achieved.

In [ ]:
#10000 trials for each set of variables
all_info = []
for trial in range(0,10000):

    for heads_or_tails in range(0,2):
        
        for basic_count in range(2,10):

            BoardState.reset()
            PGP_reached = Simulation(heads_or_tails, basic_count).runSimulation()
            PGP_reached = PGP_reached[PGP_reached['damage_counter'] >= 90].head(1)

            if not PGP_reached.empty:
                turn = PGP_reached['turn'].values[0]
                all_info.append([heads_or_tails, basic_count, turn])
            else:
                all_info.append([heads_or_tails, basic_count, '10+'])

df = pd.DataFrame(all_info, columns=['heads/tails', 'basic_count', 'turn_PGP_reached'])
display(df)

,heads/tails,basic_count,turn_PGP_reached
0,0,2,5
1,0,3,4
2,0,4,3
3,0,5,3
4,0,6,11
...,...,...,...
159995,1,5,2
159996,1,6,2
159997,1,7,3
159998,1,8,2


In [7]:
df.to_csv(r'C:\Users\Jordan\project\card-game-simulation\data.csv', index=False)